In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import scipy.optimize
import scipy.spatial
from IPython import display
from ipywidgets import interact ,widgets
import scipy.stats as stats
import math

In [2]:
# Load dataset
url = "http://datasciencemastery.in/wp-content/uploads/2018/08/titanic.csv"
train = pd.read_csv(url)
print (train.shape)

(891, 12)


In [3]:
train.pop('Cabin')
train.pop('Name')
train.pop('Ticket')


0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
5                330877
6                 17463
7                349909
8                347742
9                237736
10              PP 9549
11               113783
12            A/5. 2151
13               347082
14               350406
15               248706
16               382652
17               244373
18               345763
19                 2649
20               239865
21               248698
22               330923
23               113788
24               349909
25               347077
26                 2631
27                19950
28               330959
29               349216
             ...       
861               28134
862               17466
863            CA. 2343
864              233866
865              236852
866       SC/PARIS 2149
867            PC 17590
868              345777
869              347742
870              349248
871             

In [4]:
print (train.head())

   PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0            1         0       3    male  22.0      1      0   7.2500        S
1            2         1       1  female  38.0      1      0  71.2833        C
2            3         1       3  female  26.0      0      0   7.9250        S
3            4         1       1  female  35.0      1      0  53.1000        S
4            5         0       3    male  35.0      0      0   8.0500        S


In [5]:
# Filling missing Age values with mean
train['Age'] = train['Age'].fillna(train['Age'].mean())

# Filling missing Embarked values with most common value
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])


train['Pclass'] = train['Pclass'].apply(str)


In [6]:
# Getting Dummies from all other categorical vars
for col in train.dtypes[train.dtypes == 'object'].index:
    for_dummy = train.pop(col)
    train = pd.concat([train, pd.get_dummies(for_dummy, prefix=col)], axis=1)
    
print (train.head())    

   PassengerId  Survived   Age  SibSp  Parch     Fare  Pclass_1  Pclass_2  \
0            1         0  22.0      1      0   7.2500         0         0   
1            2         1  38.0      1      0  71.2833         1         0   
2            3         1  26.0      0      0   7.9250         0         0   
3            4         1  35.0      1      0  53.1000         1         0   
4            5         0  35.0      0      0   8.0500         0         0   

   Pclass_3  Sex_female  Sex_male  Embarked_C  Embarked_Q  Embarked_S  
0         1           0         1           0           0           1  
1         0           1         0           1           0           0  
2         1           1         0           0           0           1  
3         0           1         0           0           0           1  
4         1           0         1           0           0           1  


In [7]:
labels = train.pop('Survived')

In [8]:
# Make predictions
url = "http://datasciencemastery.in/wp-content/uploads/2018/08/test-titanic.csv"
prediction_dataset = pd.read_csv(url)

#print (prediction_dataset.head())

#X_prediction = prediction_dataset.values[:,:]   #features


prediction_dataset.pop('Cabin')
prediction_dataset.pop('Name')
prediction_dataset.pop('Ticket')

# Filling missing Age values with mean
prediction_dataset['Age'] = prediction_dataset['Age'].fillna(prediction_dataset['Age'].mean())
prediction_dataset['Fare'] = prediction_dataset['Fare'].fillna(prediction_dataset['Fare'].mean())
# Filling missing Embarked values with most common value
prediction_dataset['Embarked'] = prediction_dataset['Embarked'].fillna(prediction_dataset['Embarked'].mode()[0])
prediction_dataset['Pclass'] = prediction_dataset['Pclass'].apply(str)

# Getting Dummies from all other categorical vars
for col in prediction_dataset.dtypes[prediction_dataset.dtypes == 'object'].index:
    for_dummy = prediction_dataset.pop(col)
    prediction_dataset = pd.concat([prediction_dataset, pd.get_dummies(for_dummy, prefix=col)], axis=1)
    
#print (prediction_dataset.isnull())    
#print (prediction_dataset.head())    


# LOGISTIC REGRESSION #

In [9]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train,labels,test_size=0.25)

In [13]:
lr.fit(X_train,y_train)

D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [14]:
lr.score(X_test,y_test)

0.852017937219731

In [15]:
y_predicted=lr.predict(X_test)
from sklearn.metrics import confusion_matrix

In [16]:
cm=confusion_matrix(y_test,y_predicted)

In [17]:
cm

array([[137,  16],
       [ 17,  53]], dtype=int64)

In [18]:
cm

array([[137,  16],
       [ 17,  53]], dtype=int64)

In [19]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[17 16] [16 17] [137  53] [ 53 137]


In [20]:
n=len(FP)

In [21]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[154, 69]


In [22]:
d01=pd.DataFrame(FP,columns=['logisticregfp'])
d02=pd.DataFrame(TP,columns=['logisticregtp'])
d=pd.DataFrame(M,columns=['logisticreg'])

# DECISION TREE #

In [23]:
from sklearn import tree
model=tree.DecisionTreeClassifier()

In [24]:
model.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [25]:
model.score(X_test,y_test)

0.7802690582959642

In [26]:
y_predicted=model.predict(X_test)

In [27]:
cm=confusion_matrix(y_test,y_predicted)

In [28]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[18 31] [31 18] [122  52] [ 52 122]


In [29]:
n=len(FP)

In [30]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[140, 83]


In [31]:
d1=pd.DataFrame(M,columns=['decisiontree'])
d11=pd.DataFrame(FP,columns=['decisiontreefp'])
d12=pd.DataFrame(TP,columns=['decisiontreetp'])

# KNN(5) #

In [32]:
from sklearn.neighbors import KNeighborsClassifier

In [36]:
knn=KNeighborsClassifier(n_neighbors=5)

In [37]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [38]:
knn.score(X_test,y_test)

0.6278026905829597

In [39]:
y_predicted=knn.predict(X_test)

In [40]:
cm=confusion_matrix(y_test,y_predicted)

In [41]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[32 51] [51 32] [102  38] [ 38 102]


In [42]:
n=len(FP)

In [43]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[134, 89]


In [44]:
d2=pd.DataFrame(M,columns=['knn'])
d21=pd.DataFrame(FP,columns=['knnfp'])
d22=pd.DataFrame(TP,columns=['knntp'])

# RANDOM FOREST #

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
model=RandomForestClassifier(n_estimators=35)

In [47]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=35, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
y_predicted=model.predict(X_test)

In [49]:
model.score(X_test,y_test)

0.8251121076233184

In [50]:
cm=confusion_matrix(y_test,y_predicted)

In [51]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[18 21] [21 18] [132  52] [ 52 132]


In [52]:
n=len(FP)

In [53]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[150, 73]


In [54]:
d3=pd.DataFrame(M,columns=['rf'])
d31=pd.DataFrame(FP,columns=['rffp'])
d32=pd.DataFrame(TP,columns=['rftp'])

# SUPPORT VECTOR MACHINE #

In [55]:
from sklearn.svm import SVC

In [56]:
model=SVC(random_state=35)
model.fit(X_train,y_train)

D:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=35,
  shrinking=True, tol=0.001, verbose=False)

In [57]:
y_predicted=model.predict(X_test)

In [58]:
model.score(X_test,y_test)

0.672645739910314

In [59]:
cm=confusion_matrix(y_test,y_predicted)

In [60]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[69  4] [ 4 69] [149   1] [  1 149]


In [61]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[218, 5]


In [62]:
d4=pd.DataFrame(M,columns=['svm'])
d41=pd.DataFrame(FP,columns=['svmfp'])
d42=pd.DataFrame(TP,columns=['svmtp'])

# MLP #

In [63]:
from sklearn.neural_network import MLPClassifier

In [64]:
mlp=MLPClassifier(random_state=37)

In [65]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit(X_train).transform(X_train)
X_test_scaled=scaler.fit(X_test).transform(X_test)
mlp=MLPClassifier(max_iter=1000,random_state=42)
mlp.fit(X_train_scaled,y_train)

D:\anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
D:\anaconda\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochas

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=42, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [66]:
mlp.score(X_test_scaled,y_test)

0.8295964125560538

In [67]:
y_predicted=mlp.predict(X_test)

In [68]:
cm=confusion_matrix(y_test,y_predicted)

In [69]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[53 13] [13 53] [140  17] [ 17 140]


In [70]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[193, 30]


In [71]:
d5=pd.DataFrame(M,columns=['mlp'])
d51=pd.DataFrame(FP,columns=['mlpfp'])
d52=pd.DataFrame(TP,columns=['mlptp'])


# NAIVE BAYS #

In [72]:
import sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [73]:
bnb=BernoulliNB(binarize=0.1)
bnb.fit(X_train,y_train)
bnb.score(X_test,y_test)

0.8071748878923767

In [74]:
y_predicted=bnb.predict(X_test)

In [75]:
cm=confusion_matrix(y_test,y_predicted)

In [76]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[17 26] [26 17] [127  53] [ 53 127]


In [77]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[144, 79]


In [78]:
d6=pd.DataFrame(M,columns=['bnb'])
d61=pd.DataFrame(FP,columns=['bnbfp'])
d62=pd.DataFrame(TP,columns=['bnbtp'])
d63=pd.DataFrame(y_predicted,columns=['prd'])

In [79]:
mnb=MultinomialNB()
mnb.fit(X_train,y_train)
mnb.score(X_test,y_test)

0.7354260089686099

In [80]:
y_predicted=mnb.predict(X_test)

In [81]:
cm=confusion_matrix(y_test,y_predicted)

In [82]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[26 33] [33 26] [120  44] [ 44 120]


In [83]:
M=[]
for i in range(0,n):
    M.append(FP[i]+TP[i])
print(M)

[146, 77]


In [84]:
d7=pd.DataFrame(M,columns=['mnb'])
d71=pd.DataFrame(FP,columns=['mnbfp'])
d72=pd.DataFrame(TP,columns=['mnbtp'])
d73=pd.DataFrame(y_predicted,columns=['prd'])

In [86]:
 gnb=GaussianNB()
gnb.fit(X_train,y_train)
gnb.score(X_test,y_test)

0.8071748878923767

In [87]:
y_predicted=gnb.predict(X_test)

In [88]:
cm=confusion_matrix(y_test,y_predicted)

In [89]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(FP,FN,TP,TN)

[15 28] [28 15] [125  55] [ 55 125]
